# 1.0 - Baseline Model Comparison

## Objective

This notebook establishes baseline performance for the income prediction task. It compares a simple linear model (`ElasticNet`) against a standard ensemble model (`RandomForest`) to set a benchmark for more complex models.

This notebook will use the centralized scripts from the `src/` directory for:
- Data loading (`src/data/make_dataset.py`)
- Feature engineering (`src/features/build_features.py`)
- Model training and evaluation (`src/models/train_model.py`)
- Visualization (`src/visualization/visualize.py`)

In [ ]:
import pandas as pd
import sys
import os

# Add src to path to allow imports
sys.path.append(os.path.join(os.path.abspath(''), '..', 'src'))

from data.make_dataset import load_data
from features.build_features import (
    split_features_target,
    label_encode_features,
    one_hot_encode_features,
    split_data
)
from models.train_model import train_and_evaluate, save_model
from visualization.visualize import plot_model_evaluation

## 1. Load Data

Load the cleaned dataset from the `data/processed` directory.

In [ ]:
df = load_data('../data/processed/adult_cleaned.csv')
X, y = split_features_target(df)

df.head()

## 2. ElasticNet (Baseline Linear Model)

First, we'll train a regularized linear model. This requires `One-Hot Encoding` for the categorical variables.

In [ ]:
# Prepare data for linear models
X_ohe = one_hot_encode_features(X)
X_train_ohe, X_test_ohe, y_train, y_test = split_data(X_ohe, y)

# Train and evaluate
model_en, metrics_en = train_and_evaluate(X_train_ohe, y_train, X_test_ohe, y_test, "ElasticNet")

print("ElasticNet Performance:")
print(pd.Series(metrics_en))

In [ ]:
plot_model_evaluation(y_test, model_en, X_test_ohe, "ElasticNet", save_path_prefix='../reports/figures/1.0_elasticnet')

## 3. Random Forest (Baseline Ensemble Model)

Now, we'll train a `Random Forest` model. This is a robust ensemble method that can serve as a strong baseline. We will use `Label Encoding` for this model.

In [ ]:
# Prepare data for tree-based models
X_le = label_encode_features(X)
X_train_le, X_test_le, y_train, y_test = split_data(X_le, y)

# Train and evaluate
model_rf, metrics_rf = train_and_evaluate(X_train_le, y_train, X_test_le, y_test, "RandomForest")

print("Random Forest Performance:")
print(pd.Series(metrics_rf))

In [ ]:
plot_model_evaluation(y_test, model_rf, X_test_le, "Random Forest", save_path_prefix='../reports/figures/1.0_randomforest')

## 4. Conclusion

Let's compare the performance of the two baseline models.


In [ ]:
# Create a DataFrame for comparison
results_df = pd.DataFrame({
    "ElasticNet": metrics_en,
    "RandomForest": metrics_rf
}).round(4)

print("Baseline Model Comparison:")
print(results_df)

### Save Best Baseline Model

Based on the `ROC-AUC` score, we will select the best performing model from this baseline comparison and save it. This model will serve as the benchmark for the more advanced models in the next notebook.

In [ ]:
# Define the save path relative to the notebook's location
models_dir = '../models/'

# Determine the best model
if metrics_rf['ROC-AUC'] > metrics_en['ROC-AUC']:
    best_baseline_model = model_rf
    model_name = "RandomForest"
else:
    best_baseline_model = model_en
    model_name = "ElasticNet"

print(f"Best baseline model is: {model_name} with ROC-AUC of {results_df.loc['ROC-AUC', model_name]}")

# Construct the full path and save the model
save_path = os.path.join(models_dir, 'baseline_model.joblib')
save_model(best_baseline_model, save_path)